In [10]:
import pandas as pd
import numpy as np
from scipy.stats import norm, t, chi2
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import warnings
import time
import os
from datetime import timedelta
from vnstock import Vnstock
from arch import arch_model

warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore', category=UserWarning)

In [11]:
VN30_TICKERS = ['ACB', 'BCM', 'BID', 'BVH', 'CTG', 'FPT', 'GAS', 'GVR', 'HDB',
                'HPG', 'MBB', 'MSN', 'MWG', 'PLX', 'POW', 'SAB', 'SSI', 'STB',
                'TCB', 'TPB', 'VCB', 'VHM', 'VIB', 'VIC', 'VJC', 'VNM', 'VPB',
                'VRE', 'SSB', 'SHB']
TICKERS_TO_RUN = VN30_TICKERS 
START_DATE = '2022-01-01'
END_DATE = '2024-01-01'
ROLLING_WINDOW_SIZE = 252 
ALPHA = 0.01 
CONFIDENCE_LEVEL_PCT = int((1 - ALPHA) * 100) 
DATA_SOURCE = 'TCBS'
OUTPUT_DIR = f"vn30_var_backtesting_results_{CONFIDENCE_LEVEL_PCT}pct"
GARCH_SCALE_FACTOR = 100.0
MIN_GARCH_OBS = 60         

In [12]:
def load_and_prepare_data_vn(ticker, start_date, end_date, source=DATA_SOURCE):
    print(f"Downloading data for VN ticker: {ticker} from {start_date} to {end_date} via {source}...")
    try:
        stock = Vnstock().stock(symbol=ticker, source=source)
        df = stock.quote.history(start=start_date, end=end_date, interval='1D')
        df['Date'] = pd.to_datetime(df['time'])
        df = df.set_index('Date')
        price_col = 'close'

        prices = df[price_col].sort_index().dropna().squeeze()
        log_returns = np.log(prices / prices.shift(1)).dropna()
        print(f"Data ready for {ticker}. Log returns: {log_returns.shape}")
        return log_returns

    except AttributeError as ae: print(f"AttributeError for {ticker}: {ae}"); return None
    except Exception as e: print(f"Error loading/processing {ticker}: {e}"); return None

In [13]:
# VaR Model Implementation
def calculate_var_normal(returns, alpha=ALPHA):
    mu = np.mean(returns)
    sigma = np.std(returns)
    var = mu + sigma * norm.ppf(alpha)
    return var

def calculate_var_t(returns, alpha=ALPHA):
    if len(returns) < 3: return calculate_var_normal(returns, alpha)
    try:
        params = t.fit(returns)
        df, loc, scale = params
        if not np.isfinite(df) or df < 1 or df > 1000: return calculate_var_normal(returns, alpha)
        var = loc + scale * t.ppf(alpha, df)
        return var
    except Exception: return calculate_var_normal(returns, alpha)

def calculate_var_historical(returns, alpha=ALPHA):
    if len(returns) == 0: return np.nan
    var = np.percentile(returns, alpha * 100)
    return var

# GARCH
def fit_garch_model(returns):
    if len(returns) < MIN_GARCH_OBS:
        print(f"  Warning: Not enough observations ({len(returns)} < {MIN_GARCH_OBS}) for GARCH fit.")
        return None
    scaled_returns = returns * GARCH_SCALE_FACTOR
    try:
        am = arch_model(scaled_returns, vol='Garch', p=1, q=1, o=0, dist='normal')
        res = am.fit(update_freq=0, disp='off')
        return res
    except Exception as e:
        print(f"  Warning: GARCH(1,1) fitting failed: {e}")
        return None

# GARCH-Normal
def calculate_var_garch_normal(returns, alpha=ALPHA):
    res = fit_garch_model(returns)
    if res is None:
        return np.nan #Fail
    try:
        forecast = res.forecast(horizon=1, reindex=False)
        next_var = forecast.variance.iloc[0, 0]
        if next_var < 0 or not np.isfinite(next_var): # check forecast validity
             print("  Warning: Invalid GARCH variance forecast.")
             return np.nan
        next_sigma = np.sqrt(next_var)
        VaR_scaled = next_sigma * norm.ppf(alpha) 
        # Scale back VaR to original return scale
        VaR = VaR_scaled / GARCH_SCALE_FACTOR
        return VaR
    except Exception as e:
        print(f"Warning: Error during GARCH-Normal VaR calculation: {e}")
        return np.nan

# GARCH-T 
def calculate_var_garch_t(returns, alpha=ALPHA):
    # For GARCH-T, fit specifically with 't' distribution
    if len(returns) < MIN_GARCH_OBS: return np.nan
    scaled_returns = returns * GARCH_SCALE_FACTOR
    try:
        am_t = arch_model(scaled_returns, vol='Garch', p=1, q=1, o=0, dist='t')
        res_t = am_t.fit(update_freq=0, disp='off')
    except Exception as e:
        print(f"  Warning: GARCH(1,1)-T fitting failed: {e}")
        return np.nan 
    if res_t is None: return np.nan
    try:
        forecast_t = res_t.forecast(horizon=1, reindex=False)
        next_var_t = forecast_t.variance.iloc[0, 0]
        if next_var_t < 0 or not np.isfinite(next_var_t):
             print("  Warning: Invalid GARCH-T variance forecast.")
             return np.nan
        next_sigma_t = np.sqrt(next_var_t)
        df = res_t.params['nu']
        if not np.isfinite(df) or df <= 2:
             print(f"  Warning: Invalid GARCH-T df ({df:.2f}). Falling back to GARCH-Normal logic.")
             VaR_scaled = next_sigma_t * norm.ppf(alpha)
        else:
            t_quantile = t.ppf(alpha, df)
            # VaR_scaled = fitted_mu_t + next_sigma_t * t_quantile
            VaR_scaled = next_sigma_t * t_quantile 

        VaR = VaR_scaled / GARCH_SCALE_FACTOR
        return VaR
    except Exception as e:
        print(f"  Warning: Error during GARCH-T VaR calculation: {e}")
        return np.nan

# FHS
def calculate_var_fhs(returns, alpha=ALPHA):
    res = fit_garch_model(returns) 
    if res is None:
        return np.nan #Fail
    try:
        std_resid = (res.resid / res.conditional_volatility).dropna()
        if std_resid.empty:
            print("  Warning: No valid standardized residuals for FHS.")
            return np.nan
        forecast = res.forecast(horizon=1, reindex=False)
        next_var = forecast.variance.iloc[0, 0]
        if next_var < 0 or not np.isfinite(next_var):
             print("  Warning: Invalid GARCH variance forecast for FHS.")
             return np.nan
        next_sigma = np.sqrt(next_var)
        resid_quantile = np.percentile(std_resid, alpha * 100)
        VaR_scaled = next_sigma * resid_quantile
        VaR = VaR_scaled / GARCH_SCALE_FACTOR
        return VaR
    except Exception as e:
        print(f"  Warning: Error during FHS VaR calculation: {e}")
        return np.nan

In [15]:
def perform_backtesting(returns, var_model_func, alpha=ALPHA, window_size=ROLLING_WINDOW_SIZE):
    print(f"\nStarting backtesting for {var_model_func.__name__}...") 
    if len(returns) <= window_size:
         print(f"  Skipping backtest: Data ({len(returns)}) <= window ({window_size}).")
         return None, None, None

    backtest_start_iloc = window_size
    n_backtest = len(returns) - backtest_start_iloc
    var_estimates = np.full(n_backtest, np.nan)
    violations = np.zeros(n_backtest, dtype=int)
    actual_returns_backtest = returns.iloc[backtest_start_iloc:]

    print(f"Backtesting period: {actual_returns_backtest.index[0].date()} to {actual_returns_backtest.index[-1].date()} ({n_backtest} days)")
    print(f"Using rolling window size: {window_size} days")

    for i in range(n_backtest):
        current_estimation_window = returns.iloc[i : backtest_start_iloc + i]
        if len(current_estimation_window) != window_size: continue 

        var_est_i = var_model_func(current_estimation_window, alpha)
        var_estimates[i] = var_est_i

        if not np.isnan(var_est_i):
             actual_return_i = actual_returns_backtest.iloc[i]
             if actual_return_i < var_est_i:
                 violations[i] = 1

    var_estimates_series = pd.Series(var_estimates, index=actual_returns_backtest.index, name=f'VaR_{var_model_func.__name__}')
    violations_series = pd.Series(violations, index=actual_returns_backtest.index, name=f'Violations_{var_model_func.__name__}')

    valid_estimates_mask = ~var_estimates_series.isna()
    if not valid_estimates_mask.all():
        num_nans = sum(~valid_estimates_mask)
        print(f"  Warning: {num_nans} NaN VaR estimates encountered and excluded.")

    actual_returns_filtered = actual_returns_backtest[valid_estimates_mask]
    var_estimates_filtered = var_estimates_series[valid_estimates_mask]
    violations_filtered = violations_series[valid_estimates_mask]

    print(f"Backtesting complete for {var_model_func.__name__}. Valid observations: {len(actual_returns_filtered)}")
    return actual_returns_filtered, var_estimates_filtered, violations_filtered


In [17]:
# Metrics
def calculate_violation_ratio(violations, alpha, n_backtest):
    if n_backtest == 0: return np.nan
    n_violations = np.sum(violations)
    expected_violations = alpha * n_backtest
    if expected_violations <= 0: return np.nan
    vr = n_violations / expected_violations
    print(f"  Violations: {n_violations} observed vs. {expected_violations:.2f} expected")
    return vr

def calculate_forecast_errors(actual_returns, var_estimates, violations):
    if len(actual_returns) == 0: return np.nan, np.nan
    violations_array = violations.to_numpy() if isinstance(violations, pd.Series) else np.array(violations)
    violation_indices = np.where(violations_array == 1)[0]
    if len(violation_indices) == 0: print("  No violations. RMSE/MAPE are N/A."); return np.nan, np.nan

    actual_losses_on_violations = actual_returns.iloc[violation_indices]
    var_on_violations = var_estimates.iloc[violation_indices]
    if actual_losses_on_violations.empty: print("  Warn: No violation data for RMSE/MAPE."); return np.nan, np.nan

    errors = actual_losses_on_violations - var_on_violations
    rmse = np.sqrt(mean_squared_error(actual_losses_on_violations, var_on_violations))
    abs_actual_losses = np.abs(actual_losses_on_violations)
    valid_mape_indices = abs_actual_losses > 1e-9
    if not valid_mape_indices.any(): mape = np.nan
    else: mape_values = np.abs(errors[valid_mape_indices] / actual_losses_on_violations[valid_mape_indices]); mape = np.mean(mape_values) * 100
    print(f"  Forecast errors on {len(violation_indices)} violations.")
    return rmse, mape

def kupiec_pof_test(violations, alpha, n_backtest):
    if n_backtest == 0: return np.nan, np.nan
    n_violations = int(np.sum(violations)); n_obs = int(n_backtest)
    if alpha <= 0 or alpha >= 1: return np.nan, np.nan
    if n_violations < 0 or n_violations > n_obs: return np.nan, np.nan
    if n_violations == 0: p_value = 1.0 if alpha < 0.1 else 0.0; print(f"  Kupiec: 0 violations. Indicative p={p_value:.3f}"); return np.nan, p_value
    if n_violations == n_obs: p_value = 1.0 if alpha > 0.9 else 0.0; print(f"  Kupiec: All violations. Indicative p={p_value:.3f}"); return np.nan, p_value
    try:
        log_alpha = np.log(alpha); log_1_alpha = np.log(1 - alpha)
        p_hat = n_violations / n_obs; log_p_hat = np.log(p_hat); log_1_p_hat = np.log(1 - p_hat)
        log_likelihood_h0 = (n_obs - n_violations) * log_1_alpha + n_violations * log_alpha
        log_likelihood_h1 = (n_obs - n_violations) * log_1_p_hat + n_violations * log_p_hat
        if not (np.isfinite(log_likelihood_h0) and np.isfinite(log_likelihood_h1)): raise ValueError("Non-finite logL")
        lr_pof = -2 * (log_likelihood_h0 - log_likelihood_h1)
        if lr_pof < 0: lr_pof = 0
        p_value = chi2.sf(lr_pof, df=1)
    except (ValueError, FloatingPointError) as e: print(f"  Kupiec: Math error ({e}). NaN."); return np.nan, np.nan
    print(f"  Kupiec Test: LR={lr_pof:.3f}, p-value={p_value:.3f}")
    return lr_pof, p_value

def christoffersen_cc_test(violations, alpha, n_backtest):
    if n_backtest < 2: return np.nan, np.nan, np.nan, np.nan
    violations_arr = violations.to_numpy() if isinstance(violations, pd.Series) else np.array(violations)
    lr_pof, p_pof = kupiec_pof_test(violations, alpha, n_backtest)
    if np.isnan(lr_pof): print("  CC Test: LR_POF NaN. Cannot compute CC."); return np.nan, np.nan, np.nan, p_pof
    n_violations = int(np.sum(violations_arr))
    if n_violations == 0 or n_violations == n_backtest: print("  CC Test: Edge case. No IND test."); return np.nan, np.nan, lr_pof, p_pof
    n00, n01, n10, n11 = 0, 0, 0, 0
    for i in range(1, len(violations_arr)):
        prev, curr = int(violations_arr[i-1]), int(violations_arr[i])
        if prev == 0 and curr == 0: n00 += 1
        elif prev == 0 and curr == 1: n01 += 1
        elif prev == 1 and curr == 0: n10 += 1
        elif prev == 1 and curr == 1: n11 += 1
    n0_ = n00 + n01; n1_ = n10 + n11; _1 = n01 + n11; _0 = n00 + n10; n_trans = n_backtest - 1
    if n0_ == 0 or n1_ == 0 or _1 == 0 or n_trans == 0: print("  CC Test: Insufficient transitions."); return np.nan, np.nan, lr_pof, p_pof
    pi01 = n01 / n0_ if n0_ > 0 else 0; pi11 = n11 / n1_ if n1_ > 0 else 0; pi = _1 / n_trans
    log_likelihood_ind = 0; log_likelihood_pooled = 0
    try:
        if n00 > 0: log_likelihood_ind += n00 * np.log(1 - pi01) if (1 - pi01) > 0 else -np.inf
        if n01 > 0: log_likelihood_ind += n01 * np.log(pi01) if pi01 > 0 else -np.inf
        if n10 > 0: log_likelihood_ind += n10 * np.log(1 - pi11) if (1 - pi11) > 0 else -np.inf
        if n11 > 0: log_likelihood_ind += n11 * np.log(pi11) if pi11 > 0 else -np.inf
        if _0 > 0: log_likelihood_pooled += _0 * np.log(1 - pi) if (1 - pi) > 0 else -np.inf
        if _1 > 0: log_likelihood_pooled += _1 * np.log(pi) if pi > 0 else -np.inf
        if not np.isfinite(log_likelihood_ind) or not np.isfinite(log_likelihood_pooled): raise ValueError("Non-finite logL")
        lr_ind = -2 * (log_likelihood_pooled - log_likelihood_ind)
        if lr_ind < 0: lr_ind = 0
        p_value_ind = chi2.sf(lr_ind, df=1)
    except (ValueError, FloatingPointError) as e: print(f"  CC Test: Math error IND ({e})."); return np.nan, np.nan, lr_pof, p_pof
    lr_cc = lr_pof + lr_ind; p_value_cc = chi2.sf(lr_cc, df=2)
    print(f"  CC Test: LR_IND={lr_ind:.3f}(p={p_value_ind:.3f}), LR_CC={lr_cc:.3f}(p={p_value_cc:.3f})")
    return lr_ind, p_value_ind, lr_cc, p_value_cc

def interpret_test(p_value, significance_level=0.05):
    if isinstance(p_value, (int, float)) and not np.isnan(p_value):
        if p_value < significance_level: return f"Reject H0 (p={p_value:.3f}, Model Inadequate)"
        else: return f"Fail to Reject H0 (p={p_value:.3f}, Model Adequate)"
    elif isinstance(p_value, str) and 'N/A' in p_value: return "N/A (Test N/A)"
    elif isinstance(p_value, str) and ('Error' in p_value or 'Failed' in p_value): return p_value
    else:
        if pd.isna(p_value): return "N/A (Test N/A)"
        return f"Invalid ({p_value})"

In [20]:
# Main Function to Run Analysis
def run_multi_ticker_analysis():
    try:
        os.makedirs(OUTPUT_DIR, exist_ok=True)
        print(f"Results will be saved to directory: '{OUTPUT_DIR}'")
    except OSError as e:
        print(f"Error creating output directory '{OUTPUT_DIR}': {e}. Exiting.")
        return # Stop if directory cannot be created

    print(f"\n--- Running Analysis: {CONFIDENCE_LEVEL_PCT}% VaR ({len(TICKERS_TO_RUN)} tickers) ---")
    print(f"Tickers: {', '.join(TICKERS_TO_RUN)}")

    if not TICKERS_TO_RUN: print("No tickers. Exiting."); return
    models = {
        "Parametric (Normal)": calculate_var_normal,
        "Parametric (T-Dist)": calculate_var_t,
        "Historical Sim": calculate_var_historical,
        "GARCH(1,1)-Normal": calculate_var_garch_normal,
        "GARCH(1,1)-T": calculate_var_garch_t,
        "FHS GARCH(1,1)": calculate_var_fhs
    }

    all_ticker_results = {} 

    for ticker in TICKERS_TO_RUN:
        print(f"\n{'='*20} Processing Ticker: {ticker} {'='*20}")
        log_returns = load_and_prepare_data_vn(ticker, START_DATE, END_DATE)
        error_status = None
        if log_returns is None or log_returns.empty: error_status = "Data Error/Missing"
        elif len(log_returns) <= ROLLING_WINDOW_SIZE: error_status = "Insufficient Data"

        if error_status:
            print(f"Skipping {ticker}: {error_status}.")
            placeholder_df = pd.DataFrame({"Status": [error_status]}, index=models.keys())
            all_ticker_results[ticker] = placeholder_df
            error_filename = f"BACKTEST_DETAIL_{ticker}_{error_status.replace(' ', '_')}.csv"
            error_filepath = os.path.join(OUTPUT_DIR, error_filename)
            try: placeholder_df.to_csv(error_filepath, index=True)
            except Exception as e_save: print(f"  Warn: Save error status CSV failed: {e_save}")
            time.sleep(0.5); continue
        
       # Period for backtesting
        total_days = len(log_returns)
        days_for_backtesting = 252 # Target: approx 1 year
        required_initial_window = ROLLING_WINDOW_SIZE

        segment_start_iloc = total_days - (required_initial_window + days_for_backtesting)
        log_returns_segment = log_returns.iloc[segment_start_iloc:]

        print(f"Using data segment from {log_returns_segment.index[0].date()} for estimation + backtesting.")

        # Backtesting loop
        ticker_results_dict = {}
        backtest_actual_returns = None 

        for model_name, model_func in models.items():
            try:
                actual_ret_filtered, var_est_filtered, violations_filtered = perform_backtesting(
                    log_returns_segment, model_func, ALPHA, ROLLING_WINDOW_SIZE
                )
                if backtest_actual_returns is None and actual_ret_filtered is not None:
                    backtest_actual_returns = actual_ret_filtered
                status, n_violations, exp_violations, vr, rmse, mape = "N/A", 'N/A', 'N/A', 'N/A', 'N/A', 'N/A'
                lr_pof, p_pof, lr_ind, p_ind, lr_cc, p_cc = ('N/A',) * 6
                raw_p_pof, raw_p_cc = np.nan, np.nan

                if actual_ret_filtered is None or var_est_filtered is None or violations_filtered is None:
                    status = "Backtest Skipped/Failed"
                else:
                    n_backtest_valid = len(actual_ret_filtered)
                    if n_backtest_valid == 0:
                         status = "Backtest Valid Obs = 0"; n_violations, exp_violations = 0, 0
                    else:
                         status = "Success"
                         n_violations = violations_filtered.sum()
                         exp_violations = round(ALPHA * n_backtest_valid, 2)
                         vr = calculate_violation_ratio(violations_filtered, ALPHA, n_backtest_valid)
                         rmse, mape = calculate_forecast_errors(actual_ret_filtered, var_est_filtered, violations_filtered)
                         lr_pof, p_pof = kupiec_pof_test(violations_filtered, ALPHA, n_backtest_valid)
                         lr_ind, p_ind, lr_cc, p_cc = christoffersen_cc_test(violations_filtered, ALPHA, n_backtest_valid)
                         raw_p_pof, raw_p_cc = p_pof, p_cc # Store raw values

    
                ticker_results_dict[model_name] = {
                    "Status": status, "Violations": n_violations, "Exp. Violations": exp_violations,
                    "VR": round(vr, 3) if isinstance(vr, (int, float)) and not np.isnan(vr) else 'N/A',
                    "RMSE": round(rmse, 5) if isinstance(rmse, (int, float)) and not np.isnan(rmse) else 'N/A',
                    "MAPE": f"{mape:.2f}%" if isinstance(mape, (int, float)) and not np.isnan(mape) else 'N/A',
                    "Kupiec LR": round(lr_pof, 3) if isinstance(lr_pof, (int, float)) and not np.isnan(lr_pof) else 'N/A',
                    "Kupiec p": raw_p_pof,
                    "CC LR": round(lr_cc, 3) if isinstance(lr_cc, (int, float)) and not np.isnan(lr_cc) else 'N/A',
                    "CC p": raw_p_cc,
                }
            except Exception as e:
                 print(f"!! Runtime error backtesting {model_name} on {ticker}: {e}")
                 error_dict = {"Status": "Runtime Error", **{k: 'Error' for k in ["Violations", "Exp. Violations", "VR", "RMSE", "MAPE", "Kupiec LR", "Kupiec p", "CC LR", "CC p"]}}
                 ticker_results_dict[model_name] = error_dict
       

        #  Save
        results_df = pd.DataFrame(ticker_results_dict).T
        results_df['Kupiec Result'] = results_df['Kupiec p'].apply(interpret_test) if 'Kupiec p' in results_df else 'Error'
        results_df['CC Result'] = results_df['CC p'].apply(interpret_test) if 'CC p' in results_df else 'Error'
        if 'Kupiec p' in results_df: results_df['Kupiec p'] = results_df['Kupiec p'].apply(lambda x: f"{x:.3f}" if isinstance(x, (int, float)) and not np.isnan(x) else 'N/A')
        if 'CC p' in results_df: results_df['CC p'] = results_df['CC p'].apply(lambda x: f"{x:.3f}" if isinstance(x, (int, float)) and not np.isnan(x) else 'N/A')

        all_ticker_results[ticker] = results_df # Store final DF
        print(f"DEBUG: Stored results for {ticker}. DataFrame head:\n{all_ticker_results[ticker].head(1)}")
        print(f"DEBUG: Current keys in all_ticker_results: {list(all_ticker_results.keys())}")

        detail_filename = f"BACKTEST_DETAIL_{ticker}_{CONFIDENCE_LEVEL_PCT}pct.csv"
        detail_filepath = os.path.join(OUTPUT_DIR, detail_filename)
        try: results_df.to_csv(detail_filepath, index=True)
        except Exception as e_save: print(f"  Warn: Save detailed CSV failed: {e_save}")
        else: print(f"Saved detailed results for {ticker} to {detail_filepath}")

        time.sleep(0.5)


    print("\n" + "="*60)
    print(f"Multi-Ticker Backtesting Summary ({CONFIDENCE_LEVEL_PCT}% VaR)")
    print(f"Period: {START_DATE} to {END_DATE}, Rolling Window: {ROLLING_WINDOW_SIZE} days")
    print("="*60)

    summary_stats = {}
    model_names = list(models.keys()) # Now includes GARCH/FHS models
    total_tickers_processed = len(all_ticker_results)

    for model_name in model_names:
        successful_runs, valid_metrics_runs, total_vr, passed_kupiec, passed_cc = 0, 0, 0, 0, 0
        for ticker in all_ticker_results:
             if ticker in all_ticker_results:
                 results_df = all_ticker_results[ticker]
                 if model_name in results_df.index: 
                     res = results_df.loc[model_name]
                     if res.get("Status") not in ["Data Error/Missing", "Insufficient Data", "Runtime Error", "Backtest Skipped/Failed"]:
                         successful_runs += 1
                         if res.get("Status") == "Success":
                              valid_metrics_runs += 1
                              vr_val = res.get("VR") 
                              try: vr_float = float(vr_val); total_vr += vr_float if not np.isnan(vr_float) else 0 
                              except: pass
                              if "Fail to Reject H0" in res.get("Kupiec Result", ""): passed_kupiec += 1
                              if "Fail to Reject H0" in res.get("CC Result", ""): passed_cc += 1

        avg_vr = (total_vr / valid_metrics_runs) if valid_metrics_runs > 0 else np.nan
        perc_passed_kupiec = (passed_kupiec / valid_metrics_runs * 100) if valid_metrics_runs > 0 else np.nan
        perc_passed_cc = (passed_cc / valid_metrics_runs * 100) if valid_metrics_runs > 0 else np.nan

        summary_stats[model_name] = {
            "Attempted Runs": total_tickers_processed, "Successful Backtests": successful_runs,
            "Valid Metrics Runs": valid_metrics_runs, "Avg VR": round(avg_vr, 3) if not np.isnan(avg_vr) else 'N/A',
            "% Passed Kupiec (p>=0.05)": round(perc_passed_kupiec, 1) if not np.isnan(perc_passed_kupiec) else 'N/A',
            "% Passed CC (p>=0.05)": round(perc_passed_cc, 1) if not np.isnan(perc_passed_cc) else 'N/A'
        }

    summary_df = pd.DataFrame(summary_stats).T
    print("Overall Model Performance Across Tickers:")
    print(summary_df.to_string()) # Display full summary
    print("Notes:")
    print(" - '% Passed' calculated on runs where metrics were computable (Valid Metrics Runs).")

    summary_filename = f"BACKTEST_SUMMARY_VN30_{CONFIDENCE_LEVEL_PCT}pct.csv"
    summary_filepath = os.path.join(OUTPUT_DIR, summary_filename)
    try: summary_df.to_csv(summary_filepath, index=True)
    except Exception as e_save: print(f"  Warn: Save summary CSV failed: {e_save}")
    else: print(f"Saved summary results to {summary_filepath}")
    


In [21]:
# Run the analysis
if __name__ == "__main__":
    run_multi_ticker_analysis()

Results will be saved to directory: 'vn30_var_backtesting_results_99pct'

--- Running Analysis: 99% VaR (30 tickers) ---
Tickers: ACB, BCM, BID, BVH, CTG, FPT, GAS, GVR, HDB, HPG, MBB, MSN, MWG, PLX, POW, SAB, SSI, STB, TCB, TPB, VCB, VHM, VIB, VIC, VJC, VNM, VPB, VRE, SSB, SHB

==================== Processing Ticker: ACB ====================


2025-05-04 19:05:35 - vnstock.common.data.data_explorer - INFO - TCBS không cung cấp thông tin danh sách. Dữ liệu tự động trả về từ VCI.


Data ready for ACB. Log returns: (748,)
Using data segment from 2022-12-27 for estimation + backtesting.

Starting backtesting for calculate_var_normal...
Backtesting period: 2023-07-05 to 2023-12-29 (252 days)
Using rolling window size: 252 days
Backtesting complete for calculate_var_normal. Valid observations: 252
  Violations: 7 observed vs. 2.52 expected
  Forecast errors on 7 violations.
  Kupiec Test: LR=5.424, p-value=0.020
  Kupiec Test: LR=5.424, p-value=0.020
  CC Test: LR_IND=0.402(p=0.526), LR_CC=5.826(p=0.054)

Starting backtesting for calculate_var_t...
Backtesting period: 2023-07-05 to 2023-12-29 (252 days)
Using rolling window size: 252 days
Backtesting complete for calculate_var_t. Valid observations: 252
  Violations: 10 observed vs. 2.52 expected
  Forecast errors on 10 violations.
  Kupiec Test: LR=12.833, p-value=0.000
  Kupiec Test: LR=12.833, p-value=0.000
  CC Test: LR_IND=0.830(p=0.362), LR_CC=13.663(p=0.001)

Starting backtesting for calculate_var_historical..

2025-05-04 19:06:21 - vnstock.common.data.data_explorer - INFO - TCBS không cung cấp thông tin danh sách. Dữ liệu tự động trả về từ VCI.


Data ready for BCM. Log returns: (748,)
Using data segment from 2022-12-27 for estimation + backtesting.

Starting backtesting for calculate_var_normal...
Backtesting period: 2023-07-05 to 2023-12-29 (252 days)
Using rolling window size: 252 days
Backtesting complete for calculate_var_normal. Valid observations: 252
  Violations: 10 observed vs. 2.52 expected
  Forecast errors on 10 violations.
  Kupiec Test: LR=12.833, p-value=0.000
  Kupiec Test: LR=12.833, p-value=0.000
  CC Test: LR_IND=0.830(p=0.362), LR_CC=13.663(p=0.001)

Starting backtesting for calculate_var_t...
Backtesting period: 2023-07-05 to 2023-12-29 (252 days)
Using rolling window size: 252 days
Backtesting complete for calculate_var_t. Valid observations: 252
  Violations: 11 observed vs. 2.52 expected
  Forecast errors on 11 violations.
  Kupiec Test: LR=15.752, p-value=0.000
  Kupiec Test: LR=15.752, p-value=0.000
  CC Test: LR_IND=1.009(p=0.315), LR_CC=16.760(p=0.000)

Starting backtesting for calculate_var_histori

...

Data ready for SSB. Log returns: (748,)
Using data segment from 2022-12-27 for estimation + backtesting.

Starting backtesting for calculate_var_normal...
Backtesting period: 2023-07-05 to 2023-12-29 (252 days)
Using rolling window size: 252 days
Backtesting complete for calculate_var_normal. Valid observations: 252
  Violations: 12 observed vs. 2.52 expected
  Forecast errors on 12 violations.
  Kupiec Test: LR=18.860, p-value=0.000
  Kupiec Test: LR=18.860, p-value=0.000
  CC Test: LR_IND=1.206(p=0.272), LR_CC=20.066(p=0.000)

Starting backtesting for calculate_var_t...
Backtesting period: 2023-07-05 to 2023-12-29 (252 days)
Using rolling window size: 252 days
Backtesting complete for calculate_var_t. Valid observations: 252
  Violations: 21 observed vs. 2.52 expected
  Forecast errors on 21 violations.
  Kupiec Test: LR=53.495, p-value=0.000
  Kupiec Test: LR=53.495, p-value=0.000
  CC Test: LR_IND=3.840(p=0.050), LR_CC=57.335(p=0.000)

Starting backtesting for calculate_var_histori

The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.



Backtesting complete for calculate_var_garch_t. Valid observations: 252
  Violations: 2 observed vs. 2.52 expected
  Forecast errors on 2 violations.
  Kupiec Test: LR=0.117, p-value=0.733
  Kupiec Test: LR=0.117, p-value=0.733
  CC Test: LR_IND=0.032(p=0.858), LR_CC=0.149(p=0.928)

Starting backtesting for calculate_var_fhs...
Backtesting period: 2023-07-05 to 2023-12-29 (252 days)
Using rolling window size: 252 days
Backtesting complete for calculate_var_fhs. Valid observations: 252
  Violations: 7 observed vs. 2.52 expected
  Forecast errors on 7 violations.
  Kupiec Test: LR=5.424, p-value=0.020
  Kupiec Test: LR=5.424, p-value=0.020
  CC Test: LR_IND=0.402(p=0.526), LR_CC=5.826(p=0.054)
DEBUG: Stored results for SSB. DataFrame head:
                      Status Violations Exp. Violations     VR     RMSE  \
Parametric (Normal)  Success         12            2.52  4.762  0.02294   

                       MAPE Kupiec LR Kupiec p   CC LR   CC p  \
Parametric (Normal)  36.86%     18.8

2025-05-04 19:28:14 - vnstock.common.data.data_explorer - INFO - TCBS không cung cấp thông tin danh sách. Dữ liệu tự động trả về từ VCI.


Data ready for SHB. Log returns: (748,)
Using data segment from 2022-12-27 for estimation + backtesting.

Starting backtesting for calculate_var_normal...
Backtesting period: 2023-07-05 to 2023-12-29 (252 days)
Using rolling window size: 252 days
Backtesting complete for calculate_var_normal. Valid observations: 252
  Violations: 7 observed vs. 2.52 expected
  Forecast errors on 7 violations.
  Kupiec Test: LR=5.424, p-value=0.020
  Kupiec Test: LR=5.424, p-value=0.020
  CC Test: LR_IND=0.402(p=0.526), LR_CC=5.826(p=0.054)

Starting backtesting for calculate_var_t...
Backtesting period: 2023-07-05 to 2023-12-29 (252 days)
Using rolling window size: 252 days
Backtesting complete for calculate_var_t. Valid observations: 252
  Violations: 10 observed vs. 2.52 expected
  Forecast errors on 10 violations.
  Kupiec Test: LR=12.833, p-value=0.000
  Kupiec Test: LR=12.833, p-value=0.000
  CC Test: LR_IND=0.830(p=0.362), LR_CC=13.663(p=0.001)

Starting backtesting for calculate_var_historical..

In [22]:
#read the results from the CSV files and print them
import pandas as pd
import os
import numpy as np

RESULTS_DIR = "vn30_var_backtesting_results_99pct"
RESULTS_FILES = [f for f in os.listdir(RESULTS_DIR) if f.endswith('.csv')]

results_dict = {}
for file in RESULTS_FILES:
    file_path = os.path.join(RESULTS_DIR, file)
    ticker = file.split('_')[2] 
    try:
        df = pd.read_csv(file_path, index_col=0)
        results_dict[ticker] = df
    except Exception as e:
        print(f"Error reading {file}: {e}")
print("Results loaded from CSV files:")
for ticker, df in results_dict.items():
    print(f"\n--- Results for {ticker} ---")
    print(df.to_string())
    if ticker == 'VN30':
        df_latex = df.to_latex(index=True, float_format="%.3f")
        print("\nSummary DataFrame in LaTeX format:")
        print(df_latex)

Results loaded from CSV files:

--- Results for ACB ---
                      Status  Violations  Exp. Violations     VR     RMSE    MAPE  Kupiec LR  Kupiec p   CC LR   CC p                                Kupiec Result                                    CC Result
Parametric (Normal)  Success           7             2.52  2.778  0.01113  27.45%      5.424     0.020   5.826  0.054        Reject H0 (p=0.020, Model Inadequate)  Fail to Reject H0 (p=0.054, Model Adequate)
Parametric (T-Dist)  Success          10             2.52  3.968  0.01477  55.89%     12.833     0.000  13.663  0.001        Reject H0 (p=0.000, Model Inadequate)        Reject H0 (p=0.001, Model Inadequate)
Historical Sim       Success           5             2.52  1.984  0.01079  25.01%      1.917     0.166   2.120  0.346  Fail to Reject H0 (p=0.166, Model Adequate)  Fail to Reject H0 (p=0.346, Model Adequate)
GARCH(1,1)-Normal    Success           5             2.52  1.984  0.01143  24.81%      1.917     0.166   2.120  